In [1]:
import os
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()


'/workspace/dmc/dmc2019'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, linear_model, metrics
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, cross_validate
#from scripts.utils import own_scorer, calc_scores

In [3]:
%run notebooks/utils.ipynb

## Data

In [17]:
from sklearn.preprocessing import MinMaxScaler
df_train = pd.read_csv("data/train.csv", sep="|")
df_test = pd.read_csv("data/test.csv", sep="|")

sum_frauds, sum_non_frauds  = len(df_train[df_train.fraud == 1]), len(df_train[df_train.fraud == 0])
df_y = df_train.fraud
df_X = df_train.drop(['fraud'], axis=1)
df_X = add_new_features(df_X)
my_scaler = MinMaxScaler()
my_scaler.fit(df_X.append(df_test, sort=False))
df_X = scale_dataframe(df_X, my_scaler)
df_X.describe()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedItems
count,1879.000000,1879.000000,1879.000000,1879.000000,1879.00000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000
mean,0.480362,0.508558,0.508799,0.497266,0.49042,0.505056,0.008639,0.005327,0.067764,0.495972
std,0.341881,0.289855,0.289547,0.313743,0.31397,0.339094,0.041780,0.032800,0.120658,0.300256
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.258338,0.259680,0.181818,0.20000,0.200000,0.001175,0.000734,0.014545,0.241379
50%,0.400000,0.508475,0.512256,0.454545,0.50000,0.600000,0.002366,0.001439,0.031818,0.482759
75%,0.800000,0.762712,0.773137,0.727273,0.80000,0.800000,0.004807,0.002834,0.060606,0.758621
max,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


## Data with new Features

In [ ]:
df = df_train
df['totalScannedLineItems'] = df['scannedLineItemsPerSecond'] * df['totalScanTimeInSeconds']
df['avgTimePerScan'] = 1/ df['scannedLineItemsPerSecond']
df['avgValuePerScan'] = df['avgTimePerScan'] * df['valuePerSecond']
df['withoutRegisPerPosition'] = df['scansWithoutRegistration'] / df['totalScannedLineItems'] #equivalent to lineItemVoidsPerPosition?
df['quantiModPerPosition'] = df['quantityModifications'] / df['totalScannedLineItems']
df['lineItemVoidsPerTotal'] = df['lineItemVoids'] / df['grandTotal']
df['withoutRegisPerTotal'] = df['scansWithoutRegistration'] / df['grandTotal']
df['quantiModPerTotal'] = df['quantityModifications'] / df['grandTotal']
df['lineItemVoidsPerTime'] = df['lineItemVoids'] / df['totalScanTimeInSeconds']
df['withoutRegisPerTime'] = df['scansWithoutRegistration'] / df['totalScanTimeInSeconds']
df['quantiModPerTime'] = df['quantityModifications'] / df['totalScanTimeInSeconds']
df['valuePerScannedLineItem'] = df['valuePerSecond'] / df['scannedLineItemsPerSecond']
df1_y = df.fraud
df1_X = df.drop(['fraud'], axis=1)

In [12]:
print("Frauds: ", sum_frauds, "Non Frauds: ", sum_non_frauds)

Frauds:  104 Non Frauds:  1775


### Scoring Methods

In [13]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172)}

In [19]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(300, 450),  # default 100
    "eta" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2),
    "scale_pos_weight" : [1, sum_non_frauds/sum_frauds],
    "max_bin" : randint(200, 300)
}
default_xgb = xgb.XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_jobs=-1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=500,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='FBeta')
search.fit(df_X, df_y)
results = search.cv_results_

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 18.0min finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [20]:
scorings = {"DMC" : own_scorer, "DMC_Norm" : own_scorer_normalized}
res = cross_validate(xgbo, df_X, df_y, scoring=scorings, cv=5, n_jobs=-1)

In [27]:
df_train = pd.read_csv("data/train_new.csv", sep="|")
df_val = pd.read_csv("data/val_new.csv", sep="|")
df_train = add_new_features(df_train)
df_val = add_new_features(df_val)
a = search.best_estimator_

test_classification(a, df_train,df_val)

Results Fix Split: 
DMC Score: -155  ---  Normalized DMC Score: -0.4122340425531915, 

Results Cross Validation: 
DMC Score: 47.0  ---  Normalized DMC Score: 0.125 


{'dmc_score': -155,
 'dmc_score_norm': -0.4122340425531915,
 'cv_dmc_score': 47.0,
 'cv_dmc_score_norm': 0.125,
 'dataframe':      trustLevel  totalScanTimeInSeconds  grandTotal  lineItemVoids  \
 0             6                    1504       89.37              8   
 1             6                     128       63.32              5   
 2             3                    1675       21.22             10   
 3             1                    1753       14.41              5   
 4             5                     157       33.89             11   
 5             5                    1677       88.07              5   
 6             4                     227       71.24              0   
 7             6                     426       13.15              6   
 8             2                    1396        2.12              1   
 9             2                     585       89.34             11   
 10            6                    1612        3.34              8   
 11            2       

In [ ]:
results

In [ ]:
search.best_estimator_